In [1]:
import requests

url = "https://data.sec.gov/submissions/CIK0000320193.json"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Send a GET request to the URL with headers
response = requests.get(url, headers=headers)

if response.status_code == 200:
    json_data = response.json()
    # print(json_data)
else:
    print(f"Error: Unable to fetch data. Status code: {response.status_code}")

In [2]:
filings = json_data['filings']

In [3]:
json_data.keys()

dict_keys(['cik', 'entityType', 'sic', 'sicDescription', 'insiderTransactionForOwnerExists', 'insiderTransactionForIssuerExists', 'name', 'tickers', 'exchanges', 'ein', 'description', 'website', 'investorWebsite', 'category', 'fiscalYearEnd', 'stateOfIncorporation', 'stateOfIncorporationDescription', 'addresses', 'phone', 'flags', 'formerNames', 'filings'])

In [14]:
len(filings['recent']['accessionNumber'])

1000

In [15]:
len(filings['recent']['form'])

1000

In [16]:
filings['recent'].keys()

dict_keys(['accessionNumber', 'filingDate', 'reportDate', 'acceptanceDateTime', 'act', 'form', 'fileNumber', 'filmNumber', 'items', 'size', 'isXBRL', 'isInlineXBRL', 'primaryDocument', 'primaryDocDescription'])

In [7]:
# filings['recent']['filingDate']

In [7]:
for i,j in zip(filings['recent']['accessionNumber'],filings['recent']['form']):
    if j == '10-K': 
        print(i)
        break

0000320193-23-000106


In [13]:
# filings['recent']['form']

In [6]:
from prepline_sec_filings.fetch import (
    get_form_by_ticker, open_form_by_ticker, get_filing
)

# Fetch the 2021 10-K instead of the most recent one
rgld_cik = 320193
accession_number = "000032019323000106"
text = get_filing(rgld_cik,
                  accession_number,
                  company='Unstructured Technologies', 
                  email='support@unstructured.io')

In [11]:
print(text[1375:3284])

d04-4f75-a78c-e87ea3423788,d:1cb1ba018cb1455aa66bd3f9ab0c5b1a--><html xmlns:ix="http://www.xbrl.org/2013/inlineXBRL" xmlns:dei="http://xbrl.sec.gov/dei/2023" xmlns:ixt="http://www.xbrl.org/inlineXBRL/transformation/2020-02-12" xmlns:ixt-sec="http://www.sec.gov/inlineXBRL/transformation/2015-08-31" xmlns:aapl="http://www.apple.com/20230930" xmlns="http://www.w3.org/1999/xhtml" xmlns:link="http://www.xbrl.org/2003/linkbase" xmlns:srt="http://fasb.org/srt/2023" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:country="http://xbrl.sec.gov/country/2023" xmlns:us-gaap="http://fasb.org/us-gaap/2023" xmlns:iso4217="http://www.xbrl.org/2003/iso4217" xmlns:xbrli="http://www.xbrl.org/2003/instance" xmlns:xbrldi="http://xbrl.org/2006/xbrldi" xmlns:ecd="http://xbrl.sec.gov/ecd/2023" xml:lang="en-US"><head><meta http-equiv="Content-Type" content="text/html"/>


<title>aapl-20230930</title></head><body><div style="display:none"><ix:header><ix:hidd

In [12]:

# pipeline-api
from prepline_sec_filings.sections import section_string_to_enum, validate_section_names, SECSection
from prepline_sec_filings.sec_document import SECDocument, REPORT_TYPES, VALID_FILING_TYPES

sec_document = SECDocument.from_string(text)
risk_narrative = sec_document.get_section_narrative(SECSection.RISK_FACTORS)

for element in risk_narrative[:3]:
    print(element)
    print("\n")


The Company’s business, reputation, results of operations, financial condition and stock price can be affected by a number of factors, whether currently known or unknown, including those described below. When any one or more of these risks materialize from time to time, the Company’s business, reputation, results of operations, financial condition and stock price can be materially and adversely affected.


Because of the following factors, as well as other factors affecting the Company’s results of operations and financial condition, past financial performance should not be considered to be a reliable indicator of future performance, and investors should not use historical trends to anticipate results or trends in future periods. This discussion of risk factors contains forward-looking statements.


This section should be read in conjunction with Part II, Item 7, “Management’s Discussion and Analysis of Financial Condition and Results of Operations” and the consolidated financial state

In [1]:
from prepline_sec_filings.fetch import get_cik_by_ticker

get_cik_by_ticker('AAPL')

'0000320193'

In [2]:
cik = '0000320193'

10

In [3]:
get_cik_by_ticker('AMZN')

'0001018724'

In [4]:
get_cik_by_ticker('GOOGL')

'0001652044'

In [8]:
s = '0001652044'
int(s.strip('0'))

1652044

In [10]:
from typing import List
import re
from sec_filings import SECExtractor
import concurrent.futures
from functools import partial
from prepline_sec_filings.fetch import get_cik_by_ticker
import requests
from prepline_sec_filings.fetch import (
    get_form_by_ticker, open_form_by_ticker, get_filing
)
def sec_main(ticker:str,year:str,forms:List[str] = ['10-K','10-Q']):
    cik = get_cik_by_ticker(ticker)
    rgld_cik = int(cik.strip('0'))
    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # Send a GET request to the URL with headers
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        json_data = response.json()
    else:
        print(f"Error: Unable to fetch data. Status code: {response.status_code}")
    
    form_lists = []
    filings = json_data['filings']
    recent_filings = filings['recent']
    quarter_val = 3
    for acc_num,form_name,filing_date,report_date in zip(recent_filings['accessionNumber'],recent_filings['form'],recent_filings['filingDate'],recent_filings['reportDate']):
        if form_name in forms and filing_date.startswith(str(year)):
            if form_name == "10-Q":
                form_name+=quarter_val
                quarter_val-=1
            no_dashes_acc_num = re.sub("-","",acc_num)
            form_lists.append([no_dashes_acc_num,form_name,filing_date,report_date])
    
    acc_nums_list = [l[0] for l in form_lists]

    get_filing_partial = partial(get_filing,
                                 cik=rgld_cik,
                                company='Unstructured Technologies', 
                                email='support@unstructured.io')
    
    sec_extractor = SECExtractor(ticker=ticker)

    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        results = executor.map(get_filing_partial, acc_nums_list)
    results_texts = []
    for res in results:
        results_texts.append(res)
    assert len(results_texts) == len(acc_nums_list), f"The scraped text {len(results_texts)} is not matching with accession number texts {len(acc_nums_list)}"
    
    with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
        results = executor.map(sec_extractor.get_section_texts_from_text,results_texts)
    section_texts = []
    for res in results:
        section_texts.append(res)
    assert len(section_texts) == len(acc_nums_list), f"The section text {len(section_texts)} is not matching with accession number texts {len(acc_nums_list)}"

    for idx,val in enumerate(form_lists):
        val.append(section_texts[idx])
    return form_lists
    

In [11]:
appl_texts = sec_main('AAPL',2023)

In [21]:
from langchain.schema import Document

docs = []

for filings in appl_texts:
    # print(filings)filingDate
    texts_dict = filings[-1]
    
    for section_name, text in texts_dict.items():
        docs.append(Document(page_content=text,metadata={"accessionNumber":filings[0],"filing_type":filings[1],"filingDate":filings[2],"reportDate":filings[3],'sectionName':section_name}))

In [24]:
print({"hello":1,"there":2})

{'hello': 1, 'there': 2}


In [1]:
f = ["10-K","10-Q"]
g = ["10-K","10-Q", "10-K/A","10-Q/A"]

In [3]:
include_amends = True
k = []
if include_amends:
    for form in f:
        k.append(form)
        k.append(form+"/A")

## QUERYING DATABASE

In [5]:
from vectorDatabase import create_database
qdrant_client, encoder, speakers_list_1, speakers_list_2, speakers_list_3 = create_database("AAPL",2023)

Earnings Call Q1
Earnings Call Q2
Earnings Call Q3
SEC
Started Scraping
Scraped
Started Extracting
Extracted


In [13]:
%load_ext autoreload
%autoreload 2
from queryDatabase import query_database_earnings_call, query_database_sec

In [14]:
text = query_database_earnings_call(
    question= "What did Apple do on generative AI?", quarter="Q3", qdrant_client=qdrant_client, encoder=encoder, speakers_list = speakers_list_3
)

In [15]:
print(text)

Tim Cook: many more would be possible without AI and machine learning. And so it's absolutely critical to us. And of course, we've been doing research across a wide range of AI technologies, including generative AI for years. We're going to continue investing and innovating and responsibly advancing our products with these technologies with the goal of enriching people's lives. And so that's what it's all about for us. And as you know, we tend to announce things as they come to market, and that's our MO, and I'daround the world. From Detroit, to Naples, to Riyadh and more, we're excited to watch these talented developers embark on careers in coding and find ways to make a positive difference in their communities. Apple remains a champion of innovation, a company fueled by boundless creativity, driven by a deep sense of mission and guided by the unshakable belief that a great idea can change the world. Looking ahead, we'll continue to manage for the long term, always pushing the limits 

In [27]:
text = query_database_sec(
    question= "What did Apple do on generative AI?",
    qdrant_client=qdrant_client,encoder=encoder,
    search_form="10-Q3"
)

In [28]:
print(text)

LEGAL_PROCEEDINGS: of Appeals for the Ninth Circuit (the “Circuit Court”) affirmed the District Court’s ruling. On June 7, 2023, the Company and Epic filed petitions with the Circuit Court requesting further review of the decision. On June 30, 2023, the Circuit Court denied both petitions. On July 17, 2023, the Circuit Court granted Apple’s motion to stay enforcement of the injunction pending appeal to the U.S. Supreme Court. Epic has appealed the Circuit Court’s stay of the injunction. If the U.S. Supreme Court reverses the. stay of the injunction. If the U.S. Supreme Court reverses the Circuit Court’s stay of the injunction or declines Apple’s petition, the injunction will take effect. The Company is subject to other legal proceedings and claims that have not been fully resolved and that have arisen in the ordinary course of business. The Company settled certain matters during the third quarter of 2023 that did not individually or in the aggregate have a material impact on the Compan